In [ ]:
pip install pyvis

In [26]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('Arquivos/'):
    f.extend(filenames)
    break

In [28]:
import pandas as pd
dataframe_interacao = pd.DataFrame()

for cont, arquivo in enumerate(f):
    frame = pd.read_excel('Arquivos/' + arquivo)

    dataframe_interacao = dataframe_interacao.append(
        frame, 
        ignore_index = True
    )

C:\Users\Pedro\AppData\Local\Temp\ipykernel_8968\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8968\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8968\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.append(
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8968\1092503128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe_interacao = dataframe_interacao.appe

In [78]:
from datetime import *

data_inicial = '01/01/2021'
data_final = '01/01/2022'

data_inicial = datetime.strptime(
    data_inicial, 
    '%d/%m/%Y'
)

data_final = datetime.strptime(
    data_final, 
    '%d/%m/%Y'
)


In [79]:
relacoes = dict()

vertices = list()
incidencia_vertice = dict()

for id_issue in dataframe_interacao['ID_ISSUE'].unique():
    data_inicial_issue = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue].values[0][4]
    data_final_issue = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue].values[0][5]

    data_inicial_issue = datetime.strptime(
        data_inicial_issue, 
        '%d/%m/%Y'
    )

    data_incompleta = False

    if data_final_issue == 'Não Concluída':
        data_incompleta = True
    else:
        data_final_issue = datetime.strptime(
            data_final_issue, 
            '%d/%m/%Y'
        )
        
    
    if data_incompleta == True:
        pass
    elif data_inicial_issue < data_inicial:
        pass
    elif data_final_issue > data_final:
        pass
    else: 
        comentarios = dataframe_interacao[dataframe_interacao['ID_ISSUE'] == id_issue]
        
        usuarios_anteriores = []
        for indice, comentario in enumerate(comentarios.values):
            if indice == 0:
                usuarios_anteriores.append(comentario[3])
                
                if comentario[3] not in vertices:
                    vertices.append(comentario[3])
            
            for usuario_anterior in usuarios_anteriores:
                if comentario[6] not in vertices:
                    vertices.append(comentario[6])
                    
                if usuario_anterior > comentario[6]:
                    chave = '{}_{}'.format(usuario_anterior, comentario[6])

                    if chave not in relacoes:
                        relacoes[chave] = 1
                        
                    else:
                        relacoes[chave] = relacoes[chave] + 1
                    
                elif usuario_anterior < comentario[6]:
                    chave = '{}_{}'.format(comentario[6], usuario_anterior)

                    if chave not in relacoes:
                        relacoes[chave] = 1
                    else:
                        relacoes[chave] = relacoes[chave] + 1
                
                if comentario[6] not in usuarios_anteriores:
                    usuarios_anteriores.append(comentario[6])

In [80]:
from pyvis.network import Network

vertices_incidencia = dict()
lista_pesos = list()

net = Network('720px', '1080px')
net.barnes_hut()

for usuario in vertices:
    net.add_node(usuario)
    vertices_incidencia[usuario] = []

for relacao in list(relacoes.items()):
    usuarios = relacao[0].split('_')
    peso = relacao[1]

    usuarioA = int(usuarios[0])
    usuarioB = int(usuarios[1])

    lista_pesos.append(peso)
    vertices_incidencia[usuarioA].append(usuarioB)
    vertices_incidencia[usuarioB].append(usuarioA)

    net.add_edge(usuarioA, usuarioB, width = peso)

In [81]:
#net.show('Grafo.html')

In [82]:
total_incidencia = 0
for vertice_analisado in vertices_incidencia.values():
    total_incidencia = total_incidencia + len(vertice_analisado)

total_pesos = 0
for peso in lista_pesos:
    total_pesos = total_pesos + peso

metrica_incidencia_media = total / len(vertices_incidencia.values())
metrica_intensidade_vertices = total_pesos / len(lista_pesos)

print(metrica_incidencia_media, metrica_intensidade_vertices)
print(len(vertices_incidencia.values()))
print(len(lista_pesos))

25.14318181818182 2.734612977258502
880
4793
